In [ ]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
# from imgaug import augmenters as iaa
from scipy import ndimage
from math import ceil, floor, log
import cv2
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
import heapq
import efficientnet.tfkeras as efn 
from sklearn.model_selection import ShuffleSplit

# Set the log level of TensorFlow to suppress unnecessary output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Import necessary libraries for data preprocessing and modeling
# numpy - for numerical operations on arrays
# pandas - for data manipulation and analysis
# pydicom - for reading DICOM files
# os - for interacting with the operating system
# matplotlib - for data visualization
# collections - for specialized data structures
# tqdm - for progress bars during loops
# datetime - for working with dates and times
# scipy - for scientific computing and image processing
# math - for mathematical operations
# cv2 - for computer vision tasks
# tensorflow - for building and training machine learning models
# sys - for system-specific parameters and functions
# heapq - for heap-based data structures
# efficientnet - a library of efficient neural network architectures
# scikit-learn - for machine learning algorithms and model evaluation

# Define functions for data preprocessing and modeling here (not shown)

In [1]:
# preprocess the features  (train + val)
tbl = pd.read_csv('# The path of the training clinical features')

# combine_id + key
tbl['ID'] = tbl.apply(lambda row: str(row.hashed_patient_ir_id) + '_' + row.Hash_Key, axis = 1)
tbl = tbl.drop(['hashed_patient_ir_id', 'Hash_Key'], axis = 1)

tbl.drop_duplicates(subset=['ID'], keep='first', inplace = True)

tbl = tbl.set_index('ID')
tbl.shape

In [2]:
# preprocess the features  (train + val)
tbl_val = pd.read_csv('# The path of the validation clinical features')
tbl_val.head(5)

# combine_id + key
tbl_val['ID'] = tbl_val.apply(lambda row: str(row.hashed_patient_ir_id) + '_' + row.Hash_Key, axis = 1)
tbl_val = tbl_val.drop(['hashed_patient_ir_id', 'Hash_Key'], axis = 1)
tbl_val.drop_duplicates(subset=['ID'], keep='first', inplace = True)

tbl_val = tbl_val.set_index('ID')
tbl_val.shape

In [3]:
# preprocess testing dataset 
tbl_test = pd.read_csv('# The path of the testing clinical features')
tbl_test.head(5)

# combine_id + key
tbl_test['ID'] = tbl_test.apply(lambda row: str(row.hashed_patient_ir_id) + '_' + row.Hash_Key, axis = 1)
tbl_test = tbl_test.drop(['hashed_patient_ir_id', 'Hash_Key'], axis = 1)
tbl_test.drop_duplicates(subset=['ID'], keep='first', inplace = True)

tbl_test = tbl_test.set_index('ID')
tbl_test.shape

In [ ]:
tbl_value = pd.read_csv('# The path of the risk level labels').loc[0]

In [4]:
def _read_tbl(df, ID, desired_size):    # here the input is "int"
    try:
        row = df.loc[ID,:]
    except:
        row = tbl_value
    return row

In [ ]:
from skimage.io import imread
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def preprocess (img):
    img = img/255.0
    centered = np.subtract(img, imagenet_mean)
    standardized = np.divide(centered, imagenet_std)
    return standardized 
    return img  

def _read_img(img_dir, ID):
    img = imread(img_dir + ID)   
    img = preprocess(img)
    return img

In [ ]:
# Define the size of the input images
input_size = (224, 224)

# Define the paths to the directories containing the training, validation, and testing CXR features
train_images_dir = '# The path of the training CXR features'
val_images_dir = '# The path of the validation CXR features'
test_images_dir = '# The path of the testing CXR features'

# Define the training, validation, and testing clinical features DataFrames
train_fs_dir = tbl
valid_fs_dir = tbl_val
test_fs_dir = tbl_test

# Define the length of the clinical features vector
length = 99

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=16, dim=input_size, n_channels=3,
                 img_dir=train_images_dir, fs_dir=train_fs_dir, fs_size=length,
                 augment=False, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.img_dir = img_dir
        self.shuffle = shuffle
        self.fs_dir = fs_dir
        self.fs_size = fs_size
        self.augment = augment
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, X_fs, y = self.__data_generation(list_IDs_temp)

        return [X, X_fs], y   
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        X_fs = np.empty((self.batch_size, self.fs_size))
        y = np.empty((self.batch_size, 3), dtype=np.float32)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Read in the CXR image and clinical features for the current sample
            img = _read_img(self.img_dir, ID)
            ID_split = ID.split('-')
            X_fs[i,] = _read_tbl(self.fs_dir, ID_split[0], self.fs_size) 

            # Apply data augmentation if specified
            if self.augment:
                p = random.uniform(0,1)
                X[i,] = Augmentation(img, p)
            else:
                X[i,] = img

            # Get the label for the current sample and convert it to a one-hot encoded vector
            y_temp = self.labels.loc[ID] - 1
            y[i,:] = to_categorical(y_temp, num_classes=3)

        return X, X_fs, y

In [5]:
train_xr = '# The path of the risk level labels'
val_xr = '# The path of the risk level labels'
df = pd.read_csv(train_xr)
df = df.set_index(['ID'])
df.index

df_val = pd.read_csv(val_xr)
df_val = df_val.set_index(['ID'])
df_val.index

test_xr = '# The path of the risk level labels'

df_test = pd.read_csv(test_xr)
df_test = df_test.set_index(['ID'])
df_test.index

In [ ]:
df_test.Level.value_counts()

2    193
3    145
1    101
Name: Level, dtype: int64

In [ ]:
train_seq = DataGenerator(df.index, df, 16,input_size, 3,
                         img_dir=train_images_dir,fs_dir = train_fs_dir, fs_size = length , shuffle=True)
val_seq = DataGenerator(df_val.index, df_val, 16, input_size, 3,
                         img_dir=val_images_dir,fs_dir = valid_fs_dir, fs_size = length, shuffle=False)

test_seq = DataGenerator(df_test.index, df_test, 1, input_size, 3,
                         img_dir=test_images_dir,fs_dir = test_fs_dir, fs_size = length, shuffle=False)

In [ ]:
from tensorflow.keras.initializers import glorot_normal

# Build the base model
base_model = DenseNet121(include_top=False, weights=None, input_shape=(224,224,3))

# Add a global average pooling layer
x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(base_model.output)

# Add a dense layer
x = keras.layers.Dense(64, kernel_initializer=glorot_normal())(x)

# Apply dropout to reduce overfitting
xray = keras.layers.Dropout(0.2)(x)

# Create input layer for the clinical features
fs_in = keras.layers.Input(shape=(length,))

# Add a dense layer to process the clinical features
feature = keras.layers.Dense(128, kernel_initializer=glorot_normal())(fs_in)
feature = keras.activations.relu(feature)
feature = keras.layers.Dropout(0.2)(feature)

# Concatenate image and clinical features
fusion = keras.layers.Concatenate()([xray, feature])

# Add another dense layer to process the fusion features
fusion = keras.layers.Dense(128, kernel_initializer=glorot_normal())(fusion)
fusion = keras.layers.Activation('relu')(fusion)

# Output layer
predictions = keras.layers.Dense(3, activation="softmax", name='fusion_last')(fusion)

# Define the model
model = keras.models.Model(inputs=[base_model.input, fs_in], outputs=predictions)

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [6]:
for i in range(5):
    
    Fold = i

    # load the initial weights by name
    model.load_weights('../DenseNet_224_up_uncrop.h5', by_name = True)

    len(base_model.layers)

    for layer in model.layers[:428]:
        layer.trainable = False
    for layer in model.layers[428:]:
        layer.trainable = True


    model.compile(loss='categorical_crossentropy', 
                  optimizer=keras.optimizers.SGD(lr=0.0002, momentum=0.9,nesterov=True),
                  metrics=['acc',                       
                         keras.metrics.AUC(),
                         keras.metrics.Precision(name='precision'),
                         keras.metrics.Recall(name='recall')])


    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
    weightpath = '# The path to save the move'.format(Fold)
    es = EarlyStopping(monitor='val_acc', 
                       verbose=1, 
                       patience=8, 
                       min_delta=0.000001, 
                       mode='max')
    mc = ModelCheckpoint(weightpath, 
                         monitor='val_acc', 
                         verbose=1, 
                         save_best_only=True, 
                         mode='max')
    rlr = ReduceLROnPlateau(monitor='val_acc',
                            mode='max',
                            factor=0.1,
                            patience=3)
    
    # class weight
    classes = pd.value_counts(df.Level)
    print(classes)
    # # classes
    class_weight = [(classes[3] + classes[1] + classes[2])/classes[1], (classes[3] + classes[1] + classes[2])/classes[2],(classes[3] + classes[1] + classes[2])/classes[3]] 
    print(class_weight)
    
    
    hist = model.fit_generator(train_seq, epochs=100, verbose=1, max_queue_size=1, 
                           workers=1, validation_data=val_seq, 
                           callbacks = [es, mc,rlr], use_multiprocessing=False)  #,  class_weight = class_weight)

In [7]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

# second stage

In [1]:
for i in range(5):
    model.load_weights('The path you saved the model above.hdf5'.format(i))

    for layer in model.layers[:]:
        layer.trainable = True


    es = EarlyStopping(monitor='val_acc', 
                       verbose=1, 
                       patience=10, 
                       min_delta=0.0000001, 
                       mode='max')
    mc = ModelCheckpoint('# The path to .hdf5'.format(i), 
                         monitor='val_acc', 
                         verbose=1, 
                         save_best_only=True, 
                         mode='max')
    rlr = ReduceLROnPlateau(monitor='val_acc',
                            mode='max',
                            factor=0.1,
                            patience=5)

    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=0.00002, momentum=0.9,nesterov=True), 
                  metrics=['acc', 
                           tf.keras.metrics.AUC(name='auc'), 
                           tf.keras.metrics.Precision(name='precision'), 
                           tf.keras.metrics.Recall(name='recall')])


    hist = model.fit_generator(train_seq, epochs=100, verbose=1, max_queue_size=1, 
                               workers=1, validation_data=val_seq, 
                               callbacks = [es, mc,rlr], use_multiprocessing=False) #, class_weight = class_weight)

In [2]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [ ]:
model.load_weights('# The path where you saved the best weights')

In [ ]:
val_seq = DataGenerator(df_val.index, df_val, 1, input_size, 3,
                         img_dir=val_images_dir,fs_dir = valid_fs_dir, fs_size = length, shuffle=False)
Y_pred = model.predict_generator(val_seq,)
y_true = val_seq.labels.Level.values

Y_pred = np.argmax(Y_pred, axis = 1)
Y_pred = Y_pred + 1

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = np.round(Y_pred)
print('Confusion Matrix')
cm = confusion_matrix(y_true, y_pred)
target_names = ['0', '1', '2']
print(cm)

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_true, Y_pred)
auc = auc(fpr, tpr)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(auc))
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.title('ROC curve')
plt.legend(loc='lower right')

# Testing

In [3]:
for i in range(5):
    for j in range(1):
        if j == 0:
            model.load_weights('The path of the best model at Stage 2'.format(i))
        else:
            model.load_weights('The path of the best model at Stage 2'.format(i))

        Y_pred = model.predict_generator(test_seq)
        yy_pred = Y_pred
        Y_pred = np.argmax(Y_pred, axis = 1)
        Y_pred = Y_pred + 1

        y_true = test_seq.labels.Level.values

        from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, f1_score, roc_auc_score, precision_score
        y_pred = np.round(Y_pred)

        print('i = ', i, 'j = ', j+1)
        print(accuracy_score(y_true, y_pred))
        print(recall_score(y_true, y_pred, average = 'weighted'))
        print(precision_score(y_true, y_pred, average = 'weighted'))
        print(f1_score(y_true, y_pred, average = 'weighted'))
        print(roc_auc_score(y_true, yy_pred, average = 'weighted', multi_class = 'ovo'))
        print('**********')

# Grad-cam

In [ ]:
from tensorflow.keras.preprocessing import image
sample = '# one sample image'  
input_path = '# path of the image' + sample
input_img = image.load_img(input_path, target_size=(224, 224),
                           color_mode='rgb', interpolation='lanczos')
img_array = np.asarray(input_img, dtype='float64')
img = preprocess(img_array)
img = np.expand_dims(img, axis=0)
print(img.shape)
#tabular
fea = _read_tbl(tbl_test,sample.split('-')[0], (99,))
fea = np.expand_dims(fea, axis=0)


(1, 224, 224, 3)


In [ ]:
from keract import get_activations
import keract
activations = get_activations(model, [img, fea], layer_names = 'bn', auto_compile=True)

print(model.predict([img, fea]))

lab = df_test.loc[sample,:]
print(lab)

yy = np.zeros((1,3))
yy[:,lab-1] = 1
print(yy)
print(yy.shape)

Run it without eager mode. Paste those commands at the beginning of your script:
> import tensorflow as tf
> tf.compat.v1.disable_eager_execution()


In [ ]:
res = keract.get_gradients_of_activations(model,[img, fea] , yy, layer_names = 'avg_pool ', output_format='simple')
res['avg_pool'].shape
activations['bn'].shape

(1, 7, 7, 1024)

In [ ]:
ACT = np.zeros((7,7))
for i in range(res['avg_pool'].shape[1]):
    ACT = ACT + res['avg_pool'][0, i] * activations['bn'][0, :,:,i]
ACT[ACT < 0] = 0

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

ACT2 = NormalizeData(ACT)

In [4]:
import matplotlib.cm as cm
from IPython.display import Image, display
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))


save_and_display_gradcam(input_path, ACT2, cam_path = 'img_both.png')